### Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectgors of any size, upto ones that possibly do not fit in RAM. It also contians supporting code for evaluation and parameter tuning

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

Created a chunk of size 598, which is longer than the specified 200
Created a chunk of size 835, which is longer than the specified 200
Created a chunk of size 759, which is longer than the specified 200


In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content="Bangkok,[a] officially known in Thai as Krung Thep Maha Nakhon[b] and colloquially as Krung Thep,[c] is the capital and most populous city of Thailand. The city occupies 1,568.7 square kilometres (605.7 sq mi) in the Chao Phraya River delta in central Thailand and has an estimated population of 9.0 million as of 2021, 13% of the country's population. Over 17.4 million people (25%) lived within the surrounding Bangkok Metropolitan Region at the 2021 estimate, making Bangkok an extreme primate city, dwarfing Thailand's other urban centres in both size and importance to the national economy.[5]"),
 Document(metadata={'source': 'speech.txt'}, page_content="Bangkok traces its roots to a small trading post during the Ayutthaya Kingdom in the 15th century, which eventually grew and became the site of two capital cities, Thonburi in 1767 and Rattanakosin in 1782. Bangkok was at the heart of the modernization of Siam during the late-19t

In [5]:
embeddings = OllamaEmbeddings(model="gemma2:2b")
db = FAISS.from_documents(docs, embeddings)

In [6]:
db

In [7]:
### quering
query = "What bangkok famous for"
docs = db.similarity_search(query)
docs

[Document(metadata={'source': 'speech.txt'}, page_content="Bangkok traces its roots to a small trading post during the Ayutthaya Kingdom in the 15th century, which eventually grew and became the site of two capital cities, Thonburi in 1767 and Rattanakosin in 1782. Bangkok was at the heart of the modernization of Siam during the late-19th century, as the country faced pressures from the West. The city was at the centre of the country's political struggles throughout the 20th century, as Siam—later renamed Thailand—abolished absolute monarchy, adopted constitutional rule, and underwent numerous coups and several uprisings. The city, incorporated as a special administrative area under the Bangkok Metropolitan Administration in 1972, grew rapidly during the 1960s through the 1980s and now exerts a significant impact on Thailand's politics, economy, education, media and modern society."),
 Document(metadata={'source': 'speech.txt'}, page_content="The Asian investment boom in the 1980s and 

In [9]:
docs[0].page_content

"Bangkok traces its roots to a small trading post during the Ayutthaya Kingdom in the 15th century, which eventually grew and became the site of two capital cities, Thonburi in 1767 and Rattanakosin in 1782. Bangkok was at the heart of the modernization of Siam during the late-19th century, as the country faced pressures from the West. The city was at the centre of the country's political struggles throughout the 20th century, as Siam—later renamed Thailand—abolished absolute monarchy, adopted constitutional rule, and underwent numerous coups and several uprisings. The city, incorporated as a special administrative area under the Bangkok Metropolitan Administration in 1972, grew rapidly during the 1960s through the 1980s and now exerts a significant impact on Thailand's politics, economy, education, media and modern society."

## As a Retriever

### We can also convert the vectorstore into a retriever class. This allow us to easily use it in other LangChain methods, which largely work with retrievers.

In [11]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

"Bangkok traces its roots to a small trading post during the Ayutthaya Kingdom in the 15th century, which eventually grew and became the site of two capital cities, Thonburi in 1767 and Rattanakosin in 1782. Bangkok was at the heart of the modernization of Siam during the late-19th century, as the country faced pressures from the West. The city was at the centre of the country's political struggles throughout the 20th century, as Siam—later renamed Thailand—abolished absolute monarchy, adopted constitutional rule, and underwent numerous coups and several uprisings. The city, incorporated as a special administrative area under the Bangkok Metropolitan Administration in 1972, grew rapidly during the 1960s through the 1980s and now exerts a significant impact on Thailand's politics, economy, education, media and modern society."

## Similarity Search with score.

### There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only document but also the distance score of the query to them. The returned distance score is L2 distance. Therefore a lower score is better.

In [12]:
docs_and_score = db.similarity_search_with_score(query)

In [13]:
docs_and_score

[(Document(metadata={'source': 'speech.txt'}, page_content="Bangkok traces its roots to a small trading post during the Ayutthaya Kingdom in the 15th century, which eventually grew and became the site of two capital cities, Thonburi in 1767 and Rattanakosin in 1782. Bangkok was at the heart of the modernization of Siam during the late-19th century, as the country faced pressures from the West. The city was at the centre of the country's political struggles throughout the 20th century, as Siam—later renamed Thailand—abolished absolute monarchy, adopted constitutional rule, and underwent numerous coups and several uprisings. The city, incorporated as a special administrative area under the Bangkok Metropolitan Administration in 1972, grew rapidly during the 1960s through the 1980s and now exerts a significant impact on Thailand's politics, economy, education, media and modern society."),
  7237.0303),
 (Document(metadata={'source': 'speech.txt'}, page_content="The Asian investment boom i

In [14]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-0.5775565505027771,
 -1.102858304977417,
 -1.3160176277160645,
 -0.9950879812240601,
 1.4470959901809692,
 0.5660210847854614,
 -3.5245800018310547,
 -0.9750033020973206,
 -1.295520544052124,
 0.7846817374229431,
 -1.4118419885635376,
 -1.9566527605056763,
 -0.38378870487213135,
 -0.818576991558075,
 -1.0439026355743408,
 0.46862462162971497,
 -3.494615077972412,
 -2.009272813796997,
 -7.540505409240723,
 -1.7493077516555786,
 0.285292387008667,
 -1.5399072170257568,
 -2.0764377117156982,
 0.17753832042217255,
 2.385127067565918,
 0.22042478621006012,
 0.004329012241214514,
 -0.5351232886314392,
 1.335628628730774,
 1.3829911947250366,
 0.428800493478775,
 -0.4600939452648163,
 -0.1800810843706131,
 -0.3571873903274536,
 0.6113505363464355,
 1.595790147781372,
 -0.605991542339325,
 0.6504761576652527,
 -0.8264409303665161,
 -2.6544322967529297,
 -0.6671135425567627,
 2.182779312133789,
 0.7804065942764282,
 -0.14234919846057892,
 -0.6768156886100769,
 0.015708230435848236,
 -0.028153

In [15]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(metadata={'source': 'speech.txt'}, page_content="Bangkok traces its roots to a small trading post during the Ayutthaya Kingdom in the 15th century, which eventually grew and became the site of two capital cities, Thonburi in 1767 and Rattanakosin in 1782. Bangkok was at the heart of the modernization of Siam during the late-19th century, as the country faced pressures from the West. The city was at the centre of the country's political struggles throughout the 20th century, as Siam—later renamed Thailand—abolished absolute monarchy, adopted constitutional rule, and underwent numerous coups and several uprisings. The city, incorporated as a special administrative area under the Bangkok Metropolitan Administration in 1972, grew rapidly during the 1960s through the 1980s and now exerts a significant impact on Thailand's politics, economy, education, media and modern society."),
 Document(metadata={'source': 'speech.txt'}, page_content="The Asian investment boom in the 1980s and 

In [16]:
## saving and loading

db.save_local("faiss_index")

In [17]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)

In [18]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content="Bangkok traces its roots to a small trading post during the Ayutthaya Kingdom in the 15th century, which eventually grew and became the site of two capital cities, Thonburi in 1767 and Rattanakosin in 1782. Bangkok was at the heart of the modernization of Siam during the late-19th century, as the country faced pressures from the West. The city was at the centre of the country's political struggles throughout the 20th century, as Siam—later renamed Thailand—abolished absolute monarchy, adopted constitutional rule, and underwent numerous coups and several uprisings. The city, incorporated as a special administrative area under the Bangkok Metropolitan Administration in 1972, grew rapidly during the 1960s through the 1980s and now exerts a significant impact on Thailand's politics, economy, education, media and modern society."),
 Document(metadata={'source': 'speech.txt'}, page_content="The Asian investment boom in the 1980s and 